# Introduksjon til DuckDB Spatial

DuckDB er en analytisk database som kan kjøre in-memory eller mot filer. Med spatial-utvidelsen får vi kraftige romlige funksjoner direkte i SQL.

## Læringsmål
- Laste romlige data fra GeoParquet og DuckDB-filer
- Utføre romlige spørringer i SQL
- Gjøre romlige joiner og bufferanalyser i DuckDB
- Konvertere mellom DuckDB og GeoPandas
- Sammenligne ytelse mellom DuckDB og GeoPandas

## Data
Vi bruker de samme datasettene som i geopandas-introen:
- `buildings_sample.parquet` - Bygningsavtrykk
- `flomsoner_sample.parquet` - Flomsoner
- `sykkelrute_senterlinje_sample.parquet` - Sykkelruter

In [2]:
import subprocess

subprocess.run([
    "uv", "pip", "install",
    "duckdb",
    "geopandas",
    "pyarrow"
])

Using Python 3.12.12 environment at: /Users/alexandesn/dev/.virtualenvs/geoenv
Audited 3 packages in 18ms


CompletedProcess(args=['uv', 'pip', 'install', 'duckdb', 'geopandas', 'pyarrow'], returncode=0)

In [3]:
from pathlib import Path
import duckdb
import geopandas as gpd
import pandas as pd

# Koble til DuckDB (in-memory)
con = duckdb.connect()

# Last inn spatial-utvidelsen
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

print("DuckDB spatial klar!")
print(f"DuckDB versjon: {con.execute('SELECT version()').fetchone()[0]}")

DuckDB spatial klar!
DuckDB versjon: v1.4.3


## Last inn data fra GeoParquet

DuckDB kan lese GeoParquet-filer direkte med spatial-utvidelsen.

In [4]:
# Definer filstier
data_dir = Path("outputs/geoparquet")
buildings_path = data_dir / "buildings_sample.parquet"
flood_zones_path = data_dir / "flomsoner_sample.parquet"
cycle_routes_path = data_dir / "sykkelrute_senterlinje_sample.parquet"

# Opprett tabeller fra GeoParquet-filer
con.execute(f"CREATE TABLE buildings AS SELECT * FROM read_parquet('{buildings_path}')")
con.execute(f"CREATE TABLE flood_zones AS SELECT * FROM read_parquet('{flood_zones_path}')")
con.execute(f"CREATE TABLE cycle_routes AS SELECT * FROM read_parquet('{cycle_routes_path}')")

print("Tabeller opprettet!")
print(f"Bygninger: {con.execute('SELECT COUNT(*) FROM buildings').fetchone()[0]} rader")
print(f"Flomsoner: {con.execute('SELECT COUNT(*) FROM flood_zones').fetchone()[0]} rader")
print(f"Sykkelruter: {con.execute('SELECT COUNT(*) FROM cycle_routes').fetchone()[0]} rader")

Tabeller opprettet!
Bygninger: 10000 rader
Flomsoner: 10000 rader
Sykkelruter: 10000 rader


## Grunnleggende utforsking med SQL

In [5]:
# Vis kolonnene i buildings-tabellen
result = con.execute("DESCRIBE buildings").fetchall()
print("Kolonner i buildings:")
for row in result:
    print(f"  {row[0]}: {row[1]}")

Kolonner i buildings:
  gid: BIGINT
  osm_id: VARCHAR
  code: BIGINT
  fclass: VARCHAR
  name: VARCHAR
  type: VARCHAR
  geom: GEOMETRY


In [6]:
# Vis første radene fra hver tabell
print("Bygninger (5 første):")
print(con.execute("SELECT * FROM buildings LIMIT 5").df())

print("\nFlomsoner (5 første):")
print(con.execute("SELECT * FROM flood_zones LIMIT 5").df())

print("\nSykkelruter (5 første):")
print(con.execute("SELECT * FROM cycle_routes LIMIT 5").df())

Bygninger (5 første):
       gid     osm_id  code    fclass  name            type  \
0   302517  256981899  1500  building  None           house   
1  1113866  917527996  1500  building  None           house   
2  2741098  955796078  1500  building  None           house   
3   484043  474890912  1500  building  None           cabin   
4   921733  758449296  1500  building  None  farm_auxiliary   

                                                geom  
0  [5, 4, 0, 0, 0, 0, 0, 0, 223, 21, 141, 72, 137...  
1  [5, 4, 0, 0, 0, 0, 0, 0, 217, 111, 253, 71, 14...  
2  [5, 4, 0, 0, 0, 0, 0, 0, 194, 48, 94, 72, 191,...  
3  [5, 4, 0, 0, 0, 0, 0, 0, 157, 152, 171, 72, 72...  
4  [5, 4, 0, 0, 0, 0, 0, 0, 11, 61, 4, 72, 227, 3...  

Flomsoner (5 første):
     gid  objid    objtype  lavpunkt  gjentaksintervall  \
0  43565  43555  FlomAreal       0.0                200   
1  60009  60068  FlomAreal       0.0                 50   
2  11227  11234  FlomAreal       0.0                 10   
3  42888  

## Romlige operasjoner i DuckDB

DuckDB spatial støtter mange romlige funksjoner som ligner på PostGIS.

In [7]:
# Beregn areal for bygninger
query = """
SELECT 
    ST_Area(geom) as area,
    ST_Perimeter(geom) as perimeter,
    ST_GeometryType(geom) as geom_type
FROM buildings
LIMIT 10
"""

result = con.execute(query).df()
print("Bygninger med beregnet areal og omkrets:")
print(result)

# Totalt areal
total = con.execute("SELECT SUM(ST_Area(geom)) as total_area FROM buildings").fetchone()[0]
avg = con.execute("SELECT AVG(ST_Area(geom)) as avg_area FROM buildings").fetchone()[0]
print(f"\nTotalt areal: {total:.2f} kvadratmeter")
print(f"Gjennomsnittlig bygningsstørrelse: {avg:.2f} kvadratmeter")

Bygninger med beregnet areal og omkrets:
         area  perimeter     geom_type
0  189.915101  62.344049  MULTIPOLYGON
1  137.938047  52.800897  MULTIPOLYGON
2  157.092167  53.742298  MULTIPOLYGON
3   84.059069  40.338676  MULTIPOLYGON
4   26.618473  20.854010  MULTIPOLYGON
5   13.967478  15.119177  MULTIPOLYGON
6  164.642535  55.351554  MULTIPOLYGON
7  104.180856  44.324819  MULTIPOLYGON
8  135.975654  50.205649  MULTIPOLYGON
9   43.590009  26.493105  MULTIPOLYGON

Totalt areal: 1380043.33 kvadratmeter
Gjennomsnittlig bygningsstørrelse: 138.00 kvadratmeter


In [8]:
# Beregn sentroider og buffere
query = """
SELECT 
    ST_Centroid(geom) as centroid,
    ST_Buffer(geom, 50) as buffer_50m,
    ST_ConvexHull(geom) as convex_hull,
    ST_Envelope(geom) as envelope,
    geom
FROM buildings
LIMIT 5
"""

result = con.execute(query).df()
print("Bygninger med sentroider, buffere og hull:")
print(result[['centroid', 'buffer_50m']].head())

Bygninger med sentroider, buffere og hull:
                                            centroid  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...   

                                          buffer_50m  
0  [2, 4, 0, 0, 0, 0, 0, 0, 163, 15, 141, 72, 37,...  
1  [2, 4, 0, 0, 0, 0, 0, 0, 217, 86, 253, 71, 47,...  
2  [2, 4, 0, 0, 0, 0, 0, 0, 70, 36, 94, 72, 91, 9...  
3  [2, 4, 0, 0, 0, 0, 0, 0, 93, 146, 171, 72, 228...  
4  [2, 4, 0, 0, 0, 0, 0, 0, 140, 48, 4, 72, 127, ...  


In [9]:
# Geometriske sjekker
validity_query = """
SELECT 
    COUNT(*) as total,
    SUM(CASE WHEN ST_IsValid(geom) THEN 1 ELSE 0 END) as valid,
    SUM(CASE WHEN ST_IsEmpty(geom) THEN 1 ELSE 0 END) as empty
FROM buildings
"""

result = con.execute(validity_query).fetchone()
print(f"Geometriske egenskaper:")
print(f"Totalt: {result[0]}")
print(f"Gyldige geometrier: {result[1]} / {result[0]}")
print(f"Tomme geometrier: {result[2]}")

Geometriske egenskaper:
Totalt: 10000
Gyldige geometrier: 10000 / 10000
Tomme geometrier: 0


## Romlige joiner i DuckDB

Spatial joins gjøres med vanlig JOIN syntax kombinert med romlige predikater.

In [10]:
# Finn bygninger som overlapper flomsoner
query = """
SELECT 
    b.*,
    f.objid as flood_zone_id,
    ST_Area(b.geom) as building_area
FROM buildings b
JOIN flood_zones f ON ST_Intersects(b.geom, f.geom)
"""

buildings_in_flood = con.execute(query).df()
print(f"Bygninger som overlapper flomsoner: {len(buildings_in_flood)}")
print("\nDe første resultatene:")
print(buildings_in_flood[['building_area', 'flood_zone_id']].head())

Bygninger som overlapper flomsoner: 28

De første resultatene:
   building_area  flood_zone_id
0     631.095056          59052
1     631.095056          27896
2     631.095056          16976
3      98.344033          41242
4     186.980339          22539


In [11]:
# Beregn snittareal mellom bygninger og flomsoner
query = """
SELECT 
    b.gid as building_id,
    ST_Area(b.geom) as building_area,
    ST_Area(ST_Intersection(b.geom, f.geom)) as intersection_area,
    (ST_Area(ST_Intersection(b.geom, f.geom)) / ST_Area(b.geom) * 100) as flood_coverage_pct
FROM buildings b
JOIN flood_zones f ON ST_Intersects(b.geom, f.geom)
LIMIT 10
"""

result = con.execute(query).df()
print("Bygninger i flomsoner med dekningsprosent:")
print(result)

Bygninger i flomsoner med dekningsprosent:
   building_id  building_area  intersection_area  flood_coverage_pct
0      4139748     631.095056         631.095056          100.000000
1      4139748     631.095056         631.095056          100.000000
2      4139748     631.095056         130.042021           20.605774
3      2525622      98.344033          98.344033          100.000000
4      2786060     186.980339          37.112291           19.848232
5      2496472      86.497845          21.535394           24.897029
6      1756721     104.425546          99.407082           95.194218
7      1756721     104.425546         104.425546          100.000000
8       670287     335.953125          10.808582            3.217289
9      1993353     110.421903         110.421903          100.000000


In [12]:
# Finn sykkelruter som passerer gjennom flomsoner
query = """
SELECT 
    c.*,
    f.objid as flood_zone_id
FROM cycle_routes c
JOIN flood_zones f ON ST_Intersects(c.geom, f.geom)
"""

routes_in_flood = con.execute(query).df()
print(f"Sykkelrutesegmenter i flomsoner: {len(routes_in_flood)}")
print("\nEksempelresultater:")
print(routes_in_flood.head())

Sykkelrutesegmenter i flomsoner: 216

Eksempelresultater:
    gid     objtype skilting  anleggsnummer  uukoblingsid belysning  \
0  5949  Sykkelrute       JA           <NA>          <NA>             
1  4609  Sykkelrute       JA           <NA>          <NA>             
2  6317  Sykkelrute       JA           <NA>          <NA>             
3  5813  Sykkelrute       JA           <NA>          <NA>             
4  5813  Sykkelrute       JA           <NA>          <NA>             

                                lokalid  \
0  9fb43352-b48e-4197-b929-23c2cbee94af   
1  40e4fff4-0cd6-4ce7-8e3d-69cd9a67ce4e   
2  f98c7a88-cf7a-443e-91a2-551f3bdaee4b   
3  7f8c5808-eabc-4723-ac13-e5fef5037045   
4  7f8c5808-eabc-4723-ac13-e5fef5037045   

                                           navnerom  \
0  http://data.geonorge.no/TurruterNGIS/Turruter/so   
1  http://data.geonorge.no/TurruterNGIS/Turruter/so   
2  http://data.geonorge.no/TurruterNGIS/Turruter/so   
3  http://data.geonorge.no/TurruterN

## Avansert romlig analyse

In [13]:
# Slå sammen alle flomsoner til én geometri
query = """
SELECT 
    ST_Union_Agg(geom) as total_flood_geom,
    SUM(ST_Area(geom)) as total_flood_area
FROM flood_zones
"""

result = con.execute(query).fetchone()
print(f"Totalt flomareal: {result[1]:.2f} kvadratmeter")

# Lagre den sammenslåtte geometrien for videre bruk
con.execute("CREATE TABLE total_flood AS SELECT ST_Union_Agg(geom) as geom FROM flood_zones")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Totalt flomareal: 340028066.70 kvadratmeter


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
# Finn bygninger innen 100m fra sykkelruter
query = """
SELECT DISTINCT
    b.gid,
    ST_Area(b.geom) as area
FROM buildings b, cycle_routes c
WHERE ST_DWithin(b.geom, c.geom, 100)
"""

buildings_near_routes = con.execute(query).df()
print(f"Bygninger innen 100m fra sykkelruter: {len(buildings_near_routes)}")
print(f"Totalt areal: {buildings_near_routes['area'].sum():.2f} kvadratmeter")

Bygninger innen 100m fra sykkelruter: 486
Totalt areal: 75417.97 kvadratmeter


## Eksempel: Kompleks analyse med begge verktøy

Vi kombinerer DuckDB (for tunge spørringer) med GeoPandas (for visualisering).

In [21]:
# Bruk DuckDB til å finne bygninger i flomsoner med stor dekning
query = """
SELECT 
    b.gid,
    b.geom,
    ST_Area(b.geom) as building_area,
    ST_Area(ST_Intersection(b.geom, f.geom)) as flood_area,
    (ST_Area(ST_Intersection(b.geom, f.geom)) / ST_Area(b.geom) * 100) as flood_pct
FROM buildings b
JOIN flood_zones f ON ST_Intersects(b.geom, f.geom)
WHERE (ST_Area(ST_Intersection(b.geom, f.geom)) / ST_Area(b.geom) * 100) > 50
ORDER BY flood_pct DESC
"""

high_risk_buildings = con.execute(query).df()
print(f"Bygninger med >50% flomdekning: {len(high_risk_buildings)}")
print("\nTopp 10 mest utsatte bygninger:")
print(high_risk_buildings[['gid', 'building_area', 'flood_area', 'flood_pct']].head(10))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Bygninger med >50% flomdekning: 19

Topp 10 mest utsatte bygninger:
       gid  building_area   flood_area  flood_pct
0   745573    3559.542632  3559.542632      100.0
1   777662     200.374786   200.374786      100.0
2  4139748     631.095056   631.095056      100.0
3  2525622      98.344033    98.344033      100.0
4  1035129     255.861929   255.861929      100.0
5  4139748     631.095056   631.095056      100.0
6  2623531     102.331641   102.331641      100.0
7  1983274      58.272819    58.272819      100.0
8  1983274      58.272819    58.272819      100.0
9  1977573     191.981752   191.981752      100.0


## Oppsummering

Du har lært:
1. **DuckDB Spatial** - Laste spatial-utvidelsen og jobbe med GeoParquet
2. **Romlige funksjoner** - ST_Area, ST_Buffer, ST_Intersects, ST_Distance, osv.
3. **Spatial joins** - Kombinere tabeller basert på romlige relasjoner
4. **Aggregering** - ST_Union_Agg og grid-basert analyse

## Når bruker man hva?

**DuckDB Spatial er best for:**
- Store datasett (millioner av rader)
- Komplekse SQL-spørringer og aggregeringer
- Integrasjon med eksisterende SQL-pipelines
- Filbaserte databaser som kan deles

**GeoPandas er best for:**
- Mindre til mellomstore datasett
- Interaktiv utforskning og visualisering
- Python-orienterte arbeidsflyter
- Rask prototyping

**Best praksis:** Bruk DuckDB for tunge beregninger, deretter GeoPandas for visualisering!

In [ ]:
# Lukk DuckDB-tilkoblingen
con.close()
print("DuckDB-tilkobling lukket.")